In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import matplotlib.pyplot as plt

def dealImg(img):
    b, g, r = cv2.split(img)
    img_rgb = cv2.merge([r, g, b])
    return img_rgb

path = './'  # 原图路径
model_path = 'face_landmarker.task'

# 初始化MediaPipe
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE)
landmarker = FaceLandmarker.create_from_options(options)

# 获取图片列表
imgs_ = os.listdir(path)
imgs = [i for i in imgs_ if '2' in i]

plt.figure(figsize=(5184/96, 3888/96), dpi=96)

for img_list in imgs:
    cropped_save_path = 'face_blur_/' + img_list
    imgs_list = os.listdir(img_list)
    for img in imgs_list:
        img_path = os.path.join(img_list, img)
        mp_image = mp.Image.create_from_file(img_path)
        face_landmarker_result = landmarker.detect(mp_image)
        mp_width = mp_image.width
        mp_height = mp_image.height
        image = cv2.imread(img_path)
        
        if not face_landmarker_result.face_landmarks:
            print(img_path)
            continue
            
        landmarks = face_landmarker_result.face_landmarks[0]
        
        # 第一步模糊处理（来自第一个文件）
        left_up_x = int(landmarks[53].x * mp_width)
        left_up_y = int(landmarks[53].y * mp_height)
        right_down_x = int(landmarks[346].x * mp_width)
        right_down_y = int(landmarks[346].y * mp_height)
        eye_region_width = right_down_x - left_up_x
        eye_region_height = int(0.375 * eye_region_width)
        fix_ycenter = int((right_down_y + left_up_y) * 0.5)
        fix_left_up_y = fix_ycenter - int(0.5 * eye_region_height)
        
        mask = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.float32)
        face_point = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377,
                     152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109]
        
        face_mask = []
        for i in face_point:
            face_mask.append([int(landmarks[i].x * mp_width), int(landmarks[i].y * mp_height)])
     
        face_mask_temp = cv2.fillConvexPoly(mask, np.int32(face_mask), (1.0, 1.0, 1.0))

        mask += face_mask_temp
        mask[fix_left_up_y:fix_left_up_y + eye_region_height, left_up_x:left_up_x + eye_region_width] = 0
        mask[fix_left_up_y + eye_region_height:int(landmarks[2].y*mp_height), int(landmarks[129].x*mp_width):int(landmarks[358].x*mp_width)] = 0
        
        mask = 255 * np.uint8(mask)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 40))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, 1)
        mask = cv2.GaussianBlur(mask, (15, 15), cv2.BORDER_DEFAULT)
        
        inverseMask = cv2.bitwise_not(mask)
        mask = mask.astype(float)/255 
        inverseMask = inverseMask.astype(float)/255
        faces = cv2.medianBlur(image, 149)

        faces = faces.astype(float)/255 
        ladyFace = image.astype(float)/255
        
        justeyes = cv2.multiply(mask, faces)
        justFace = cv2.multiply(inverseMask, ladyFace)

        first_result = justFace + justeyes
        
        # 确保第一步结果在0-1范围内
        first_result = np.clip(first_result, 0, 1)
        
        # 第二步模糊处理（来自第二个文件，在第一处理结果基础上）
        mask2 = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.float32)
        mask2[fix_left_up_y + eye_region_height:int(landmarks[2].y * 1.05 * mp_height), 
              int(landmarks[129].x * 0.95*mp_width):int(landmarks[358].x * 1.05*mp_width)] = 1
        
        mask2 = 255 * np.uint8(mask2)
        mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernel, 1)
        mask2 = cv2.GaussianBlur(mask2, (15, 15), cv2.BORDER_DEFAULT)
        
        inverseMask2 = cv2.bitwise_not(mask2)
        mask2 = mask2.astype(float)/255 
        inverseMask2 = inverseMask2.astype(float)/255
        
        # 将第一步结果转换为uint8进行模糊处理
        first_result_uint8 = (first_result * 255).astype(np.uint8)
        faces2 = cv2.medianBlur(first_result_uint8, 99)
        faces2 = faces2.astype(float)/255 
        
        # 使用第一步结果作为背景
        ladyFace2 = first_result
        
        justeyes2 = cv2.multiply(mask2, faces2)
        justFace2 = cv2.multiply(inverseMask2, ladyFace2)
        
        final_result = justFace2 + justeyes2
        final_result = np.clip(final_result, 0, 1)  # 确保最终结果在0-1范围内
    
        plt.imshow(dealImg((final_result * 255).astype(np.uint8)), 'gray')
        plt.axis('off')
        plt.savefig(os.path.join(cropped_save_path, img), dpi=96, bbox_inches='tight', pad_inches=0)
        plt.clf()

<Figure size 5184x3888 with 0 Axes>